In [1]:
%pip install flask SQLAlchemy sqlite3

In [2]:
from flask import Flask, request, jsonify 
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker

In [3]:
engine = create_engine("sqlite:///:memory:", echo=False)
Base = declarative_base()
SessionLocal = sessionmaker(bind=engine)

In [4]:
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)

Base.metadata.create_all(engine)

In [5]:
app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

@app.route("/users", methods=["GET", "POST"])
def users():
    session = SessionLocal()
    if request.method == "POST":
        data = request.get_json(force=True)
        user = User(name=data["name"])
        session.add(user)
        session.commit()
        return jsonify({"message": f"User {user.name} added!", "id": user.id})
    else:  # GET
        all_users = session.query(User).all()
        return jsonify([{"id": u.id, "name": u.name} for u in all_users])

In [6]:
import io
from werkzeug.test import EnvironBuilder
from werkzeug.wrappers import Request

def call_wsgi(app, path="/", method="GET", json=None):
    builder = EnvironBuilder(path=path, method=method)
    if json:
        import json as pyjson
        data = pyjson.dumps(json)
        builder.input_stream = io.BytesIO(data.encode("utf-8"))
        builder.headers["Content-Type"] = "application/json"
        builder.headers["Content-Length"] = str(len(data))
    env = builder.get_environ()

    response = []
    def start_response(status, headers, exc_info=None):
        response.append(status)
        response.append(headers)

    result = app(env, start_response)
    body = b"".join(result).decode("utf-8")
    status, headers = response[0], dict(response[1])
    return status, headers, body


In [7]:
# Call hello route
print(call_wsgi(app, "/"))

# Add a user
print(call_wsgi(app, "/users", method="POST", json={"name": "Alice"}))

# Add another
print(call_wsgi(app, "/users", method="POST", json={"name": "Bob"}))

# List users
print(call_wsgi(app, "/users", method="GET"))


('200 OK', {'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '20'}, '<p>Hello, World!</p>')
('200 OK', {'Content-Type': 'application/json', 'Content-Length': '39'}, '{"id":1,"message":"User Alice added!"}\n')
('200 OK', {'Content-Type': 'application/json', 'Content-Length': '37'}, '{"id":2,"message":"User Bob added!"}\n')
('200 OK', {'Content-Type': 'application/json', 'Content-Length': '48'}, '[{"id":1,"name":"Alice"},{"id":2,"name":"Bob"}]\n')
